In [9]:
# required libraries
import os
import cv2
from PIL import Image
import numpy as np
import shutil



In [10]:
# Read the image, convert it to grayscale, and binarize it (convert to black and white)

def read_and_binarize_image(image_path, threshold=127):

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    image = cv2.imread(image_path)  # Read the image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    _, binary_image = cv2.threshold(gray_image, threshold, 1, cv2.THRESH_BINARY)  # Binarize (0 or 1)
    return binary_image.tolist()

In [11]:
# Save a 2D grid (list of lists) as an image
def save_grid_as_image(grid, filename):
    array = np.array(grid, dtype=np.uint8) * 255  # Convert 0/1 to 0/255 for image
    image = Image.fromarray(array)
    image.save(filename)

In [12]:
# Process the connected lines in the image
# This function returns a list of 2D grids, each representing a line in the image
def process_connected_lines(image_grid):
    # Compute the histogram (number of foreground pixels in each row)
    histogram = [sum(value != 0 for value in row) for row in image_grid]
    line_grids = []
    start_index = None

    for i, count in enumerate(histogram):
        if count > 0 and start_index is None:  # Start of a new line
            start_index = i
        elif count == 0 and start_index is not None:  # End of a line
            line_grids.append(image_grid[start_index:i])
            start_index = None

    if start_index is not None:  # Handle the last line
        line_grids.append(image_grid[start_index:])

    # Save each line as an image
    for idx, grid in enumerate(line_grids):
        save_grid_as_image(grid, f"line/line{idx + 1}.png")

    return line_grids

In [13]:
# Extract connected words from each line and save them
def process_words_in_lines(line_grids):
    for line_idx, line_grid in enumerate(line_grids):
        # Compute the histogram (number of foreground pixels in each column)
        word_histogram = [sum(row[col] != 0 for row in line_grid) for col in range(len(line_grid[0]))]
        word_grids = []
        start_col = None

        for col, count in enumerate(word_histogram):
            if count > 0 and start_col is None:  # Start of a new word
                start_col = col
            elif count == 0 and start_col is not None:  # End of a word
                word_grids.append([[row[col] for col in range(start_col, col)] for row in line_grid])
                start_col = None

        if start_col is not None:  # Handle the last word in the line
            word_grids.append([[row[col] for col in range(start_col, len(line_grid[0]))] for row in line_grid])

        # Save each word as an image
        for word_idx, grid in enumerate(word_grids):
            save_grid_as_image(grid, f"words/line{line_idx + 1}_word{word_idx + 1}.png")


In [14]:
# Flood-fill algorithm to find all connected components
def get_neighbouring_points(image, r, c):
    stack = [(r, c)]  # Stack to keep track of points to process
    points = []  # List to store connected points

    while stack:
        x, y = stack.pop()

        if image[x][y] != 1:  # Skip if not a foreground pixel
            continue

        image[x][y] = 2  # Mark the pixel as visited
        points.append((x, y))  # Add the pixel to the connected component

        # Check neighbors (3x3 neighborhood)
        for i in range(-1, 2):
            for j in range(-1, 2):
                nx, ny = x + i, y + j
                if 0 <= nx < len(image) and 0 <= ny < len(image[0]) and image[nx][ny] == 1:
                    stack.append((nx, ny))

    return points


In [15]:
# Convert connected points to a cropped image
def convert_to_sub_image(points):
    min_x = min(p[0] for p in points)
    max_x = max(p[0] for p in points)
    min_y = min(p[1] for p in points)
    max_y = max(p[1] for p in points)

    # Create a blank image for the connected component
    new_image = [[0] * (max_y - min_y + 1) for _ in range(max_x - min_x + 1)]

    for x, y in points:
        new_image[x - min_x][y - min_y] = 1  # Mark foreground pixels

    return new_image

In [16]:
# Extract and save all connected components
def process_connected_components_custom(image_grid):
    rows, cols = len(image_grid), len(image_grid[0])
    components = []

    for r in range(rows):
        for c in range(cols):
            if image_grid[r][c] == 1:
                points = get_neighbouring_points(image_grid, r, c)
                components.append(convert_to_sub_image(points))

    # Save each connected component as an image
    for idx, component in enumerate(components):
        save_grid_as_image(component, f"cc/cc{idx + 1}.png")

In [17]:
# Create folders to store line, word, and connected component images
os.makedirs("line", exist_ok=True)
os.makedirs("words", exist_ok=True)
os.makedirs("cc", exist_ok=True)

In [21]:
image_path = "image1.png"  # Path to the input image
binary_grid = read_and_binarize_image(image_path)  # Read and binarize the image

line_grids = process_connected_lines(binary_grid)  # Extract lines
process_words_in_lines(line_grids)  # Extract words
process_connected_components_custom(binary_grid)  # Extract connected components